# Import Libraries

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# Data Collection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths to the dataset directories
train_path = '/content/drive/MyDrive/forestfiredetection/Training'
valid_path = '/content/drive/MyDrive/forestfiredetection/Testing'


# Data Processing

In [ ]:
# Define image size and batch size
IMAGE_SIZE = [224, 224]



# ResNet Implementation

In [ ]:

# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights


input_shape = (224, 224, 3)
resnet = ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)


94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
# don't train existing weights(freeze layers )
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
  # useful for getting number of output classes
folders = glob('Datasets/train/*')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)


# Load Model

In [ ]:
# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:

# view the structure of the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# Pre-processing(Model,Data)

In [ ]:
# tell the model what cost and optimization method to use
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Variable Creation

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/forestfiredetection/forestfiredetection/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'binary')

Found 1547 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/forestfiredetection/forestfiredetection/Testing',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'binary')

Found 0 images belonging to 0 classes.


# Model Training

In [ ]:
'''# fit the model
r = model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set)
)
'''


# Implementation

In [ ]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

# Define image dimensions and other parameters
img_width, img_height = 224, 224
batch_size = 16
num_epochs = 10

# Load the ResNet50 model with pre-trained weights
resnet = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze the layers in ResNet50
for layer in resnet.layers:
    layer.trainable = False

# Create a new model by adding a few layers on top of ResNet50
model = Sequential([
    resnet,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/forestfiredetection/forestfiredetection/Training',
    target_size=(224,224),
    batch_size=10,
    class_mode='binary'
)

# Train the model
history = model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=10
)

# Load and preprocess an image for prediction
uploaded_image_path = '/content/fire_0015.jpg'
img = image.load_img(uploaded_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = preprocess_input(img_array)
img_array = np.expand_dims(img_array, axis=0)

# Make a prediction
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    result = "Fire"
else:
    result = "No Fire"

print("Predicted Class:", result)
print("Probability:", prediction[0])
